In [249]:
import pandas as pd

In [250]:
df = pd.read_csv("충청남도 천안시_시내버스운수업체별노선현황_20220923.csv", encoding = 'CP949')
df

,연번,노선번호,운수업체명,1회운행거리(km_편도기준),1일운행횟수(회_편도기준),비고,데이터기준일자
0,1,1,"보성,삼안,새천안",21.3,125,NaN,2022-09-23
1,2,2,"보성,삼안,새천안",23.3,67,편도,2022-09-23
2,3,3,"보성,삼안,새천안",21.8,111,NaN,2022-09-23
3,4,5,"보성,삼안,새천안",20.3,53,NaN,2022-09-23
4,5,7,"보성,삼안,새천안",23.0,59,편도,2022-09-23
...,...,...,...,...,...,...,...
154,155,870,"보성,삼안,새천안",5.7,34,전체휴지,2022-09-23
155,156,900,"보성,삼안,새천안",19.9,75,NaN,2022-09-23
156,157,910,"보성,삼안,새천안",20.0,38,NaN,2022-09-23
157,158,931,"보성,삼안,새천안",18.8,20,전체휴지,2022-09-23


# 1회운행거리, 1일운행횟수 컬럼 각 상하위 10개 버스 노선 추출

In [251]:
# onetime_distance 상위 10개

df1 = df.groupby('노선번호').agg(onetime_distance = ('1회운행거리(km_편도기준)', 'max'),
                            perday_number = ('1일운행횟수(회_편도기준)', 'max')).sort_values(['onetime_distance', 'perday_number']
                                                                                   ,ascending = False)
df1.head(10)

# 532, 381, 380, 382, 383, 166, 540, 165, 640, 391

,onetime_distance,perday_number
노선번호,,
532,43.2,4
381,40.5,6
380,40.4,2
382,40.0,6
383,39.6,12
166,35.4,1
540,34.0,10
165,32.6,10
640,32.0,10


In [252]:
# onetime_distance 하위 10개

df2 = df.groupby('노선번호').agg(onetime_distance = ('1회운행거리(km_편도기준)', 'max'),
                            perday_number = ('1일운행횟수(회_편도기준)', 'max')).sort_values(['onetime_distance', 'perday_number']
                                                                                   ,ascending = True)
df2.head(10)

# 462, 850, 662, 651, 840, 431, 99, 93, 98, 115

,onetime_distance,perday_number
노선번호,,
462,3.0,8
850,3.8,6
662,3.8,10
651,4.0,8
840,4.3,14
431,4.5,2
99,4.6,1
93,4.9,1
98,5.0,1


In [253]:
# perday_number 상위 10개

df3 = df.groupby('노선번호').agg(onetime_distance = ('1회운행거리(km_편도기준)', 'max'),
                            perday_number = ('1일운행횟수(회_편도기준)', 'max')).sort_values(['perday_number', 'onetime_distance']
                                                                                   ,ascending = False)
df3.head(10)

# 12, 14, 13, 1, 11, 400, 81, 3, 90, 20

,onetime_distance,perday_number
노선번호,,
12,14.8,172
14,16.5,155
13,12.8,143
1,21.3,125
11,17.4,118
400,23.4,113
81,13.9,113
3,21.8,111
90,11.7,101


In [254]:
# perday_number 하위 10개

df4 = df.groupby('노선번호').agg(onetime_distance = ('1회운행거리(km_편도기준)', 'max'),
                            perday_number = ('1일운행횟수(회_편도기준)', 'max')).sort_values(['perday_number', 'onetime_distance']
                                                                                   ,ascending = True)
df4.head(10)

# 99, 93, 98, 91, 590, 104, 492, 373, 105, 702

,onetime_distance,perday_number
노선번호,,
99,4.6,1
93,4.9,1
98,5.0,1
91,7.2,1
590,13.9,1
104,19.8,1
492,20.3,1
373,22.3,1
105,22.4,1


In [287]:
df4.to_clipboard(index=False)

# RFM 가중치 계산

## 최적 가중치를 찾기 위한 함수 및 사전 준비

In [272]:
import pandas as pd
rfm_score = pd.read_csv('충청남도 천안시_시내버스운수업체별노선현황_20220923.csv',  encoding = 'CP949') ## 데이터 불러오기


## 필요 변수 추출
rfm_score = rfm_score[['노선번호', '1회운행거리(km_편도기준)', '1일운행횟수(회_편도기준)']]


import pandas as pd
import numpy as np
 
from tqdm import tqdm


def get_score(level, data, reverse = False):
    

    score = [] 
    for j in range(len(data)): 
        for i in range(len(level)): 
            if data[j] <= level[i]: 
                score.append(i+1) 
                break 
            elif data[j] > max(level): 
                score.append(len(level)+1) 
                break 
            else: 
                continue
    if reverse:
        return [len(level)+2-x for x in score]
    else:
        return score 
 
grid_number = 100 ## 눈금 개수, 너무 크게 잡으면 메모리 문제가 발생할 수 있음.
weights = []
for j in range(grid_number+1):
    weights += [(i/grid_number,j/grid_number,(grid_number-i-j)/grid_number)
                  for i in range(grid_number+1-j)]
num_class = 5 ## 클래스 개수
class_level = np.linspace(1,5,num_class+1)[1:-1] ## 클래스를 나누는 지점을 정한다.
total_amount_of_sales = rfm_score['1회운행거리(km_편도기준)'].sum() ## 1회운행거리 총합 = 총 운행거리
print(class_level)




[1.8 2.6 3.4 4.2]


## 최적 가중치를 찾는 코드 

In [285]:
max_std = 0 ## 표준편차 초기값
for w in tqdm(weights,position=0,desc = '[Finding Optimal weights]'):
    ## 주어진 가중치에 따른 1회운행거리별 점수 계산
    score = w[0]*rfm_score['노선번호'] + \
                        w[1]*rfm_score['1회운행거리(km_편도기준)'] + \
                        w[2]*rfm_score['1일운행횟수(회_편도기준)'] 
    rfm_score['Class'] = get_score(class_level,score,True) ## 점수를 이용하여 고객별 등급 부여
    ## 등급별로 1회운행거리를 집계한다.
    grouped_rfm_score = rfm_score.groupby('Class')['1회운행거리(km_편도기준)'].sum().reset_index()

    ## 제약조건 추가 - 등급이 높은 노선들의 1회운행거리가 낮은 등급의 노선들보다 커야한다.
    grouped_rfm_score = grouped_rfm_score.sort_values('Class')
    
    temp_monetary = list(grouped_rfm_score['1회운행거리(km_편도기준)'])
    if temp_monetary != sorted(temp_monetary,reverse=True):
        continue
    
    
    ## 클래스별 1회운행거리를 총구매금액으로 나누어 클래스별 매출 기여도 계산
    grouped_rfm_score['1회운행거리(km_편도기준)'] = grouped_rfm_score['1회운행거리(km_편도기준)'].map(lambda x : x/total_amount_of_sales)
    std_sales = grouped_rfm_score['1회운행거리(km_편도기준)'].std() ## 매출 기여도의 표준편차 계산
    if max_std <= std_sales:
        max_std = std_sales ## 표준편차 최대값 업데이트
        optimal_weights = w  ## 가중치 업데이트

[Finding Optimal weights]: 100%|██████████████████████████████████████████████████| 5151/5151 [00:48<00:00, 105.63it/s]


In [286]:
print(optimal_weights)

(0.01, 0.61, 0.38)


## 최적 가중치 결과
- 노선번호,	1회운행거리(km_편도기준), 1일운행횟수(회_편도기준)	순으로
- (0.01, 0.61, 0.38)

# 최적 가중치 계산 결과값을 통해 버스 노선 이용량 상하위 10개 추출

- 1회운행거리가 길고, 1일 운행횟수도 많은 상위 10개 순위 추출
- 이동 교통량이 많은 지역 (버스 이용량 승하차 인원 많다)

In [303]:
df2 = df1.assign(total = df1['onetime_distance'] * 0.61 + df1['perday_number'] * 0.38).sort_values('total', ascending=False)
df2.head(10)

,onetime_distance,perday_number,total
노선번호,,,
12,14.8,172,74.388
14,16.5,155,68.965
13,12.8,143,62.148
1,21.3,125,60.493
400,23.4,113,57.214
3,21.8,111,55.478
11,17.4,118,55.454
201,31.1,87,52.031
81,13.9,113,51.419


- 12, 14, 13, 1, 400, 3, 11, 201, 81, 20 

- 1회운행거리가 짧고, 1일 운행횟수도 작은 상위 10개 순위 추출
- 이동 교통량이 적은 지역 (버스 이용량 승하차 인원 적다)

In [302]:
# 가중치 파생변수 만들기

df2 = df1.assign(total = df1['onetime_distance'] * 0.61 + df1['perday_number'] * 0.38).sort_values('total', ascending=True)

df2.head(10)

,onetime_distance,perday_number,total
노선번호,,,
99,4.6,1,3.186
93,4.9,1,3.369
98,5.0,1,3.430
431,4.5,2,3.505
850,3.8,6,4.598
91,7.2,1,4.772
462,3.0,8,4.870
651,4.0,8,5.480
231,6.7,4,5.607


- 99, 93, 98, 431, 850, 91, 462, 651, 231, 662